# `pandas` - Missing Values

Goal of this notebook is to create a useful dataset with appropriate values and variables.

__[EDIT]__

Deal with missing values

Create features (aka variables or columns)
- changing units
- splitting/joining strings

## Introduction

There are three basic steps to cleaning data:
1. Dealing with missing values
1. Manipulating strings 
1. Manipulating datetimes

Each is described in a separate section below.

First though libraries are loaded and define the `read_dataframe` function.

Load the libraries.

In [5]:
import pandas  as pd
import numpy   as np
(pd.__version__,
 np.__version__
)

In [6]:
def read_college_scorecard(file_path='/dbfs/mnt/datalab-datasets/college-scorecard/MERGED2013_PP.csv',
                           n_rows=None):
  from shutil import copyfile
  from pathlib import Path
  cache_file_path = '/tmp/MERGED2013_PP.csv'
  cache_file_obj  = Path(cache_file_path)
  if not cache_file_obj.exists():
    copyfile(file_path,
             cache_file_path)
  return pd.read_csv(cache_file_path,
                     nrows=n_rows,
                     na_values=['PrivacySuppressed']
                     )\
            .rename(columns = {'\ufeffUNITID':'UNITID'})
def read_college_scorecard(file_path='/dbfs/mnt/datalab-datasets/college-scorecard/MERGED2013_PP.csv',
                           n_rows=None):
  from shutil import copyfile
  import os.path
  cache_file_path = '/tmp/MERGED2013_PP.csv'
  if not os.path.exists(cache_file_path):
    copyfile(file_path,
             cache_file_path)
  return pd.read_csv(cache_file_path,
                     nrows=n_rows,
                     na_values=['PrivacySuppressed']
                     )\
            .rename(columns = {'\ufeffUNITID':'UNITID'})


In [7]:
read_college_scorecard().info()

In [8]:
read_college_scorecard()\
  .select_dtypes(include=['object'])\
  .head()

## Missing Values

Recall that the `isnull` method takes a Series/DataFrame as input and indicates whether the elements of the input are `NaN`. 

The output has the same shape as the input.

In [11]:
(read_college_scorecard().shape,
 read_college_scorecard().isnull().shape
)

The first five values of the `C150_L4_POOLED_SUPP` variable are missing.

In [13]:
read_college_scorecard().isnull().head()

This can be used to list the variables/columns with missing values and indicate the number of missing values for each.

In [15]:
read_college_scorecard().isnull().sum(axis=0).sort_values(ascending=False)[lambda x: x > 0]

This list has many elements so a visual inspection isn't very useful. 

The `describe` method summarizes the number of missing values of the variables.

In [17]:
read_college_scorecard().isnull().sum(axis=0).sort_values(ascending=False)[lambda x: x > 0].describe()

Notice that over half of the variables contain no valid (non-missing) values. 

In the next sections:
- Columns will be dropped that do not meet a certain threshold of valid values.
- Columns will be dropped that do not meet a certain threshold of valid values.
- The remaining missing values will be imputed (filled in.)

### Drop Rows/Columns (`dropna` Method)
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.dropna.html

Notice the number of rows and columns in the dataframe.

In [21]:
read_college_scorecard()\
  .shape

Drop all columns with fewer than 90% (7024) valid values,

In [23]:
read_college_scorecard()\
  .dropna(thresh=7024,axis=1)\
  .shape

List the columns that remain.

In [25]:
read_college_scorecard()\
  .dropna(thresh=7024,axis=1)\
  .pipe(lambda df: list(df.columns))

Drop all rows with fewer than 90% (254) valid values.

In [27]:
read_college_scorecard()\
  .dropna(thresh=7024,axis=1)\
  .dropna(thresh= 254,axis=0)\
  .shape

Notice that there are now `7086` rows after dropping rows with fewer than `254` valid values. (We had `7804` rows.)

List the number of remaining missing values for all variables. In all cases we should have fewer than 10% (780) missing values.

In [30]:
read_college_scorecard()\
  .dropna(thresh=7024,axis=1)\
  .dropna(thresh= 254,axis=0)\
  .isnull()\
  .sum(axis=0)\
  .sort_values(ascending=False)[lambda x: x > 0]

### Replace Missing Values (`fillna` Method)

- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.fillna.html

Filling forward/backward, which is often used to fill missing values in time series, is not covered here.

Which variable should not be imputed? The default imputation method is the median of the valid values. Consider whether there are variables  that:
1. require a different imputation method
1. should never be missing or imputed

The second case implies that rows with missing values for that variable should be dropped.

Check below for variables with missing values (to impute.)

In [35]:
read_college_scorecard()\
  .dropna(thresh=7024,axis=1)\
  .dropna(thresh= 254,axis=0)\
  .isnull()\
  .sum(axis=0)\
  .sort_values(ascending=False)[lambda x: x > 0]

Check the datatypes of the columns with missing values.

In [37]:
read_college_scorecard()\
  .dropna(thresh=7024,axis=1)\
  .dropna(thresh= 254,axis=0)\
  .pipe(lambda df: df.loc[:,df.isnull().any()])\
  .dtypes

As a first attempt I tried using values as 'median'. This silently didn't work. Notice below.

In [39]:
NPCURL_ndx = read_college_scorecard()\
               .dropna(thresh=7024,axis=1)\
               .dropna(thresh= 254,axis=0)\
               .loc[:,'NPCURL']\
               .isnull()
NPCURL_ndx

In [40]:
read_college_scorecard()\
  .dropna(thresh=7024,axis=1)\
  .dropna(thresh= 254,axis=0)\
  .fillna(value='median')\
  .loc[NPCURL_ndx,['NPCURL','UGDS']]

Replacing the `object` variables `NPCURL` and `INSTURL` with the string `NA` seems reasonable, replacing it with the mode does not seem reasonable. The code below does just that.

In [42]:
read_college_scorecard()\
  .dropna(thresh=7024,axis=1)\
  .dropna(thresh= 254,axis=0)\
  .fillna(value={'NPCURL': 'NA', 
                 'INSTURL': 'NA'})\
  .loc[NPCURL_ndx,['NPCURL','UGDS']]

The remaining variables are numeric and will be filled by their (respective) medians, but each variable has a different median.

This command creates a dictionary whose keys are variables with missing values and values are the medians (of the key variable.) These medians are calculated by ignoring the missing values.

In [44]:
median_dict = \
  read_college_scorecard()\
    .dropna(thresh=7024,axis=1)\
    .dropna(thresh= 254,axis=0)\
    .fillna(value={'NPCURL': 'NA', 
                   'INSTURL': 'NA'})\
    .pipe(lambda df: df.loc[:,df.isnull().any()])\
    .apply(lambda x: x.median(skipna=True),axis=0)\
    .to_dict()
(median_dict['UGDS'], 
 median_dict
)

The `median_dict` dictionary is used as the `value` parameter below.

In [46]:
median_dict = \
  read_college_scorecard()\
    .dropna(thresh=7024,axis=1)\
    .dropna(thresh= 254,axis=0)\
    .fillna(value={'NPCURL': 'NA', 
                   'INSTURL': 'NA'})\
    .pipe(lambda df: df.loc[:,df.isnull().any()])\
    .apply(lambda x: x.median(skipna=True),axis=0)\
    .to_dict()
read_college_scorecard()\
  .dropna(thresh=7024,axis=1)\
  .dropna(thresh= 254,axis=0)\
  .fillna(value={'NPCURL': 'NA',
                 'INSTURL': 'NA'})\
  .fillna(value=median_dict)\
  .loc[NPCURL_ndx,['NPCURL','UGDS']]

Check that there are no remaining missing values.

In [48]:
median_dict = \
  read_college_scorecard()\
    .dropna(thresh=7024,axis=1)\
    .dropna(thresh= 254,axis=0)\
    .fillna(value={'NPCURL': 'NA', 
                   'INSTURL': 'NA'})\
    .pipe(lambda df: df.loc[:,df.isnull().any()])\
    .apply(lambda x: x.median(skipna=True),axis=0)\
    .to_dict()
read_college_scorecard()\
  .dropna(thresh=7024,axis=1)\
  .dropna(thresh= 254,axis=0)\
  .fillna(value={'NPCURL': 'NA',
                 'INSTURL': 'NA'})\
  .fillna(value=median_dict)\
  .isnull()\
  .sum(axis=0)\
  .sort_values(ascending=False)

## Final Code

The code that reads the dataset and deals with missing values is contained in the cell below.

In [50]:
def read_college_scorecard(file_path='/dbfs/mnt/datalab-datasets/college-scorecard/MERGED2013_PP.csv',
                           n_rows=None):
  from shutil import copyfile
  from pathlib import Path
  cache_file_path = '/tmp/MERGED2013_PP.csv'
  cache_file_obj  = Path(cache_file_path)
  if not cache_file_obj.exists():
    copyfile(file_path,
             cache_file_path)
  return pd.read_csv(cache_file_path,
                     nrows=n_rows,
                     na_values=['PrivacySuppressed']
                     )\
            .rename(columns = {'\ufeffUNITID':'UNITID'})
def read_college_scorecard(file_path='/dbfs/mnt/datalab-datasets/college-scorecard/MERGED2013_PP.csv',
                           n_rows=None):
  from shutil import copyfile
  import os.path
  cache_file_path = '/tmp/MERGED2013_PP.csv'
  if not os.path.exists(cache_file_path):
    copyfile(file_path,
             cache_file_path)
  return pd.read_csv(cache_file_path,
                     nrows=n_rows,
                     na_values=['PrivacySuppressed']
                     )\
            .rename(columns = {'\ufeffUNITID':'UNITID'})

def fix_missing_values(df):
  median_dict = \
    read_college_scorecard()\
      .dropna(thresh=7024,axis=1)\
      .dropna(thresh= 254,axis=0)\
      .fillna(value={'NPCURL': 'NA', 
                     'INSTURL': 'NA'})\
      .pipe(lambda df: df.loc[:,df.isnull().any()])\
      .apply(lambda x: x.median(skipna=True),axis=0)\
      .to_dict()
  return df.dropna(thresh=7024,axis=1)\
           .dropna(thresh= 254,axis=0)\
           .fillna(value={'NPCURL': 'NA',
                          'INSTURL': 'NA'})\
           .fillna(value=median_dict)

These functions are used as:

In [52]:
read_college_scorecard()\
  .pipe(fix_missing_values)

There's always something. See warning below.

In [54]:
x = read_college_scorecard().pipe(fix_missing_values)

In [55]:
x.columns[[7,9]]

In [56]:
x.iloc[:,[7,9]].dtypes

In [57]:
x.iloc[:,[7,9]].head()

__The End__